In [1]:
import torch
from torch import nn
import numpy as np 
from math import sqrt

In [2]:
input_size = 16
batch_size = 13
target_size = 17
hidden_size = 8
n_head = 2
single_dim = hidden_size // n_head
query = torch.FloatTensor(batch_size, input_size).uniform_(0, 1)

In [3]:
target = torch.FloatTensor(batch_size, target_size, input_size).uniform_(0, 1)

In [4]:
W_q = nn.Linear(input_size, hidden_size, bias=False)
W_k = nn.Linear(input_size, hidden_size, bias=False)
W_v = nn.Linear(input_size, hidden_size, bias=False)
W_out = nn.Linear(hidden_size, input_size, bias=False)

In [5]:
q_c = W_q(query)
k = W_k(target)
v = W_v(target)

In [6]:
q_c.shape, k.shape, v.shape

(torch.Size([13, 8]), torch.Size([13, 17, 8]), torch.Size([13, 17, 8]))

In [7]:
v_r = v.reshape(batch_size, target_size, n_head, single_dim).permute(0, 2, 1, 3) 
#batch_size x n_head x target_size x single_head_dim

In [13]:
ret1 = torch.einsum("ik,ijk->ijk", [q_c,k])
ret1 = ret1.reshape(batch_size, target_size, n_head, single_dim).sum(-1) / sqrt(single_dim)
ret1 = ret1.permute(0, 2, 1) #batch_size x n_head x query_size x target_size
ret1.shape
r_softmax = ret1.softmax(-1)
r_softmax.shape


torch.Size([13, 2, 17])

In [203]:
r_softmax

tensor([[[0.0595, 0.0577, 0.0603, 0.0567, 0.0598, 0.0578, 0.0573, 0.0600,
          0.0598, 0.0574, 0.0585, 0.0587, 0.0599, 0.0585, 0.0605, 0.0583,
          0.0594],
         [0.0543, 0.0621, 0.0597, 0.0586, 0.0613, 0.0554, 0.0556, 0.0580,
          0.0556, 0.0488, 0.0582, 0.0569, 0.0597, 0.0640, 0.0610, 0.0612,
          0.0696]],

        [[0.0562, 0.0588, 0.0631, 0.0614, 0.0546, 0.0552, 0.0613, 0.0534,
          0.0592, 0.0614, 0.0605, 0.0620, 0.0643, 0.0558, 0.0540, 0.0610,
          0.0577],
         [0.0557, 0.0615, 0.0602, 0.0613, 0.0550, 0.0577, 0.0595, 0.0560,
          0.0549, 0.0621, 0.0601, 0.0585, 0.0592, 0.0588, 0.0613, 0.0583,
          0.0600]],

        [[0.0579, 0.0586, 0.0588, 0.0583, 0.0589, 0.0591, 0.0592, 0.0577,
          0.0584, 0.0600, 0.0592, 0.0586, 0.0597, 0.0595, 0.0582, 0.0602,
          0.0579],
         [0.0612, 0.0594, 0.0597, 0.0578, 0.0576, 0.0604, 0.0593, 0.0561,
          0.0568, 0.0580, 0.0596, 0.0561, 0.0582, 0.0581, 0.0620, 0.0605,
          0.0

In [204]:
v_r.shape, r_softmax.shape
_ret = torch.matmul(v_r.transpose(-1, -2), r_softmax.unsqueeze(-1)).squeeze(-1)
_ret.shape

torch.Size([13, 2, 4])

In [209]:
_ret = _ret.reshape(batch_size, -1)

In [210]:
_ret.shape

torch.Size([13, 8])

In [212]:
res = W_out(_ret)

In [213]:
_res.permute(0, 2, 1, 3).sum(-2).shape

torch.Size([13, 3, 16])

In [214]:
_res

tensor([[[[ 1.0731e-01,  5.8213e-01,  3.3198e-01,  ...,  1.5870e-01,
            2.5539e-01,  3.3690e-01],
          [ 1.0836e-01,  5.8805e-01,  3.3941e-01,  ...,  1.5994e-01,
            2.6216e-01,  3.3867e-01],
          [ 1.0827e-01,  5.8202e-01,  3.3503e-01,  ...,  1.5979e-01,
            2.5627e-01,  3.3735e-01]],

         [[-1.1920e-01, -3.8302e-02,  2.5744e-01,  ..., -5.7536e-02,
           -1.4551e-01,  1.4655e-01],
          [-1.1925e-01, -3.8628e-02,  2.5704e-01,  ..., -5.7621e-02,
           -1.4569e-01,  1.4673e-01],
          [-1.1975e-01, -3.9917e-02,  2.5667e-01,  ..., -5.8352e-02,
           -1.4713e-01,  1.4751e-01]]],


        [[[ 6.7252e-02,  6.6533e-01,  3.4646e-01,  ...,  1.6072e-01,
            1.8352e-01,  4.2381e-01],
          [ 6.7363e-02,  6.6733e-01,  3.4545e-01,  ...,  1.6095e-01,
            1.8326e-01,  4.2508e-01],
          [ 6.7806e-02,  6.6182e-01,  3.4423e-01,  ...,  1.6016e-01,
            1.8385e-01,  4.2103e-01]],

         [[-1.7025e-01, -6.05

In [14]:
class MultiHeadAttention(nn.Module):
    def __init__(self,
                 query_dim,
                 key_dim,
                 num_units,
                 dropout_p=0.1,
                 h=8,
                 is_masked=False):
        super(MultiHeadAttention, self).__init__()

        if query_dim != key_dim:
            raise ValueError("query_dim and key_dim must be the same")
        if num_units % h != 0:
            raise ValueError("num_units must be dividable by h")
        if query_dim != num_units:
            raise ValueError("to employ residual connection, the number of "
                             "query_dim and num_units must be the same")

        self._num_units = num_units
        self._h = h
        self._key_dim = float(key_dim)**-0.5
        self._dropout_p = dropout_p
        self._is_masked = is_masked

        self.query_layer = nn.Linear(query_dim, num_units, bias=False)
        self.key_layer = nn.Linear(key_dim, num_units, bias=False)
        self.value_layer = nn.Linear(key_dim, num_units, bias=False)
        self.proj_layer = nn.Linear(num_units, num_units)
        self.ln = nn.LayerNorm(query_dim)

    def forward(self, query, keys):
        """
        Args:
            query (torch.Tensor): [seq_len, batch, embed_dim]
            keys (torch.Tensor): [seq_len, batch, embed_dim]
        Returns:
            torch.Tensor: [seq_len, batch, embed_dim]
        """
        Q = self.query_layer(query)
        K = self.key_layer(keys)
        V = self.value_layer(keys)

        batch_size = query.size(0)
        seq_len = query.size(1)
        # split each Q, K and V into h different values from dim 2
        # and then merge them back together in dim 0
        chunk_size = int(self._num_units / self._h)
        Q = Q.view(batch_size * self._h, seq_len, chunk_size)
        K = K.view(batch_size * self._h, -1, chunk_size)
        V = V.view(batch_size * self._h, -1, chunk_size)

        # calculate QK^T
        attention = torch.bmm(Q, K.transpose(1, 2))
        # normalize with sqrt(dk)
        attention = attention * self._key_dim
        # use masking (usually for decoder) to prevent leftward
        # information flow and retains auto-regressive property
        # as said in the paper
        if self._is_masked:
            diag_vals = attention[0].sign().abs()
            diag_mat = diag_vals.tril(abs(Q.size(1) - K.size(1)))
            diag_mat = diag_mat.unsqueeze(0).expand(attention.size())
            mask = torch.ones(
                diag_mat.size(), device=query.device) * (-2**32 + 1)
            # this is some trick that I use to combine the lower diagonal
            # matrix and its masking. (diag_mat-1).abs() will reverse the value
            # inside diag_mat, from 0 to 1 and 1 to zero. with this
            # we don't need loop operation andn could perform our calculation
            # faster
            attention = (attention * diag_mat) + (mask * (diag_mat - 1).abs())
        # put it to softmax
        attention = F.softmax(attention, dim=-1)
        # apply dropout
        attention = F.dropout(
            attention, p=self._dropout_p, training=self.training)
        # multiplyt it with V
        attention = torch.bmm(attention, V)
        # convert attention back to its input original size
        attention = attention.view(batch_size, seq_len, -1)

        # apply  projection
        attention = self.proj_layer(attention.view(-1, attention.size(-1)))
        attention = attention.view(batch_size, seq_len, -1)
        # residual connection
        attention += query
        # apply layer normalization
        attention = self.ln(attention)

        return attention

    def init_weight(self):
        nn.init.uniform_(self.query_layer.weight, -0.1, 0.1)
        nn.init.uniform_(self.key_layer.weight, -0.1, 0.1)
        nn.init.uniform_(self.value_layer.weight, -0.1, 0.1)
        nn.init.uniform_(self.proj_layer.weight, -0.1, 0.1)


In [15]:
target.shape

torch.Size([13, 17, 16])

In [18]:
mh = MultiHeadAttention()

NameError: name 'input_dim' is not defined